<a href="https://colab.research.google.com/github/Manan1811/fifa20-ratings/blob/master/FIFA_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import keras
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
path = "/content/drive/My Drive/FIFA_20.csv"
fifa= pd.read_csv(path)
fifa.head()


,overall,name,nationality,club,position,preferred_foot,work_rate,jersey_number,wage_eur,international_reputation,weak_foot,skill_moves,age,height_cm,weight_kg,pace,shooting,passing,dribbling,defending,physic,finishing,fk_accuracy,acceleration,agility,stamina,penalties,composure,standing_tackle,sliding_tackle
0,94,L. Messi,Argentina,FC Barcelona,Midfielder,Left,Medium,10,565000,5,4,4,32,170,72,87,92,92,96,39,66,95,94,91,93,75,75,96,37,26
1,93,Cristiano Ronaldo,Portugal,Juventus,Midfielder,Right,High,7,405000,5,4,5,34,187,83,90,93,82,89,35,78,94,76,89,87,85,85,95,32,24
2,92,Neymar Jr,Brazil,Paris Saint-Germain,Midfielder,Right,High,10,290000,5,5,5,27,175,68,91,85,87,95,32,58,87,87,94,96,81,90,94,26,29
3,91,E. Hazard,Belgium,Real Madrid,Midfielder,Right,High,7,470000,4,4,4,28,175,74,91,83,86,94,35,66,84,79,94,95,84,88,91,27,22
4,91,K. De Bruyne,Belgium,Manchester City,Midfielder,Right,High,17,370000,4,5,4,28,181,70,76,86,92,86,61,78,82,83,77,78,89,79,91,58,51


In [6]:
skill_cols = [col for col in fifa.columns if col not in ('name', 'nationality', 'club', 'overall', 'position', 
                                                         'preferred_foot', 'work_rate', 'jersey_number', 
                                                         'international_reputation', 'weak_foot', 'skill_moves', 
                                                         'wage_eur', 'age', 'height_cm', 'weight_kg')]
fifa1= pd.read_csv(path, usecols=skill_cols)
fifa1.head()

,pace,shooting,passing,dribbling,defending,physic,finishing,fk_accuracy,acceleration,agility,stamina,penalties,composure,standing_tackle,sliding_tackle
0,87,92,92,96,39,66,95,94,91,93,75,75,96,37,26
1,90,93,82,89,35,78,94,76,89,87,85,85,95,32,24
2,91,85,87,95,32,58,87,87,94,96,81,90,94,26,29
3,91,83,86,94,35,66,84,79,94,95,84,88,91,27,22
4,76,86,92,86,61,78,82,83,77,78,89,79,91,58,51


#DEEP Learning Model